In [5]:
import lineage_graph as lg
import pandas as pd
from py2neo import authenticate, Graph, Node, Relationship


graph = lg.build_lineage_graph('loggedfs_events.log')


# Sanity check query.
query = """
    MATCH (n)
    WITH n, rand() AS random
    ORDER BY random
    OPTIONAL MATCH (n)-[r]->(m)
    RETURN n AS source_node,
           id(n) AS source_id,
           r,
           m AS target_node,
           id(m) AS target_id
"""

        
results_df = pd.DataFrame(graph.data(query))
print(results_df)

                              r  source_id  \
0  {'timestamp': 1485918018082}         59   
1                          None         61   
2  {'timestamp': 1485918018082}         60   

                                         source_node  target_id  \
0                 {'name': '/home/lubuntu/test.csv'}       60.0   
1            {'name': '/home/lubuntu/test_copy.csv'}        NaN   
2  {'name': 'cp /home/lubuntu/test.csv /home/lubu...       61.0   

                                         target_node  
0  {'name': 'cp /home/lubuntu/test.csv /home/lubu...  
1                                               None  
2            {'name': '/home/lubuntu/test_copy.csv'}  


In [6]:
from scripts.vis import draw

options = {"Dataset": "name", "Job": "uid"}
draw(graph, options)